In [225]:
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
import xgboost as xgb
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

In [226]:
import pandas as pd
import numpy as np
import zipfile
from sklearn.model_selection import KFold
import lightgbm as lgb

# 构建预测size1和size3的数据集

In [227]:
# 读取data_spc数据

In [228]:
size_train=pd.read_csv('任务A\任务A数据集\训练集\Size.csv')
df_train_spc=pd.read_csv('任务A\任务A数据集\训练集\data_spc.csv')
df_test_spc=pd.read_csv('任务A\任务A数据集\测试集\data_spc.csv')


variables1=['Sensor1', 'Sensor2', 'Sensor3', 'IJ', 'Sensor5',
       'Sensor6', 'MouldTemp1', 'MouldTemp2', 'MouldTemp3', 'MouldTemp4',
       'MouldTemp5', 'MouldTemp9', 'MouldTemp10', 'MouldTemp11', 'MouldTemp12',
       'MouldTemp13', 'MouldTemp14', 'Sensor8', 'MouldFlow1', 'MouldFlow2',
       'MouldFlow3', 'SP']

In [229]:
# 训练集高频数据特征提取
TRAIN_ZIP=zipfile.ZipFile('任务A\任务A数据集\训练集\传感器高频数据.zip')
file_list=TRAIN_ZIP.namelist()
feature_n = len(variables1)
features_ = np.empty([len(file_list), feature_n])
times_ = []
mold_id_ = []

def feature_columns(variables):
    f_cols = []
    for v in variables:
        f_cols.append(v + '_mean')
    return f_cols

def stage_features(df, variables):
    avg = []
    tmp_df = df.loc[:, variables]
    for v in variables:
        tmp_avg = tmp_df[v].mean()
        avg.append(tmp_avg)    
    return np.array(avg)

feature_n = len(variables1)
features_ = np.empty([len(file_list), feature_n])
times_ = []
mold_id_ = []
for i,f in enumerate(file_list):
    df=pd.read_csv(TRAIN_ZIP.open(f))
    tmp = f.split('_')
    ti = tmp[2]
    mold_id = tmp[3].replace('.csv', '')
    times_.append(str(ti))
    mold_id_.append(int(mold_id))
    
    if len(df) == 0:
        features_[i] = [None for j in range(feature_n)]
    else:
        features_[i] = stage_features(df, variables1)
f_cols = feature_columns(variables1)
TRAIN_HIG = pd.DataFrame(features_, columns=f_cols)
TRAIN_HIG['Time'] = times_
TRAIN_HIG['Id'] = mold_id_
TRAIN_HIG = TRAIN_HIG[['Id', 'Time'] + f_cols]
TRAIN_ZIP.close()

In [230]:
# 测试集高频数据特征提取
TEST_ZIP=zipfile.ZipFile('任务A\任务A数据集测试集\传感器高频数据.zip')
file_list=TEST_ZIP.namelist()
feature_n = len(variables1)
features_ = np.empty([len(file_list), feature_n])
times_ = []
mold_id_ = []
for i,f in enumerate(file_list):
    df=pd.read_csv(TEST_ZIP.open(f))
    tmp = f.split('_')
    ti = tmp[2]
    mold_id = tmp[3].replace('.csv', '')
    times_.append(str(ti))
    mold_id_.append(int(mold_id))
    if len(df) == 0:
        features_[i] = [None for j in range(feature_n)]
    else:
        features_[i] = stage_features(df, variables1)
f_cols = feature_columns(variables1)
TEST_HIG = pd.DataFrame(features_, columns=f_cols)
TEST_HIG['Time'] = times_
TEST_HIG['Id'] = mold_id_
TEST_HIG = TEST_HIG[['Id', 'Time'] + f_cols]
TEST_ZIP.close()

TRAIN_HIG.rename(columns={'Time':'spcTime'}, inplace=True)
TEST_HIG.rename(columns={'Time':'spcTime'}, inplace=True)

df_train_spc['spcTime']=df_train_spc['spcTime'].apply(int)
df_train_spc['spcTime']=df_train_spc['spcTime'].apply(str)
df_test_spc['spcTime']=df_test_spc['spcTime'].apply(int)
df_test_spc['spcTime']=df_test_spc['spcTime'].apply(str)

df_TRAIN=TRAIN_HIG.merge(df_train_spc, how='outer', on=['Id', 'spcTime']) 
df_TEST=TEST_HIG.merge(df_test_spc, how='outer', on=['Id', 'spcTime']) 
X_col=[i for i in df_TRAIN.columns if not i in ['Id', 'spcTime']]

In [231]:
x_train=df_TRAIN[X_col]
x_test=df_TEST[X_col]
y_train = pd.read_csv('Train\Size.csv')

In [232]:
# 训练集缺失值处理

In [233]:
x_train.isnull()

,Sensor1_mean,Sensor2_mean,Sensor3_mean,IJ_mean,Sensor5_mean,Sensor6_mean,MouldTemp1_mean,MouldTemp2_mean,MouldTemp3_mean,MouldTemp4_mean,...,EL_PP_STG_POS_4,EL_CLAMP_PRESS,EL_MAX_INJ_PRESS,EL_IV_END_PRESS,EL_CF_END_PRESS,EL_PLST_BP_0,EL_PLST_BP_1,EL_PLST_BP_2,EL_PLST_BP_3,EL_PLST_BP_4
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16595,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
16596,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
16597,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
16598,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [234]:
x_train.dropna(axis="columns", how='all', inplace=True)

In [235]:
x_train.drop("remark", axis=1, inplace = True)

In [236]:
x_train.loc[:, :] = x_train[:].fillna(0)

In [237]:
# 测试集缺失值处理

In [238]:
x_test.dropna(axis="columns", how='all', inplace=True)

In [239]:
x_test.drop("remark", axis=1, inplace = True)

In [240]:
x_test.loc[:, :] = x_test[:].fillna(0)

In [241]:
# 转换成矩阵

In [242]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)

# 预测尺寸1

In [243]:
# 利用sklearn的SelectKBest筛选出x_train特征

In [244]:
from sklearn.feature_selection import SelectKBest, f_regression

In [245]:
skb=SelectKBest(f_regression, k=21)

In [246]:
x_train=skb.fit_transform(x_train,y_train[:,0])

In [247]:
x_train.shape

(16600, 21)

In [248]:
# 筛选出x_test特征

In [249]:
mask=skb.get_support(indices=True)

In [250]:
mask

array([ 0,  1,  2,  3, 11, 12, 14, 15, 16, 23, 39, 44, 45, 47, 48, 80, 84,
       85, 91, 93, 94], dtype=int64)

In [251]:
x_test

array([[27.00725206, 38.78191556, 34.80156381, ...,  5.        ,
         5.        ,  5.        ],
       [26.51587619, 38.68803619, 33.52371746, ...,  5.        ,
         5.        ,  5.        ],
       [25.80545905, 38.42890667, 37.67869333, ...,  5.        ,
         5.        ,  5.        ],
       ...,
       [27.25748598, 32.62649844,  5.00346978, ...,  5.        ,
         5.        ,  5.        ],
       [26.2822405 , 31.75218318,  5.00226231, ...,  5.        ,
         5.        ,  5.        ],
       [24.97500062, 31.03079439,  5.0468    , ...,  5.        ,
         5.        ,  5.        ]])

In [252]:
x_test=x_test[:,mask]

In [253]:
x_test.shape

(3971, 21)

In [254]:
x_test

array([[27.00725206, 38.78191556, 34.80156381, ..., 46.818726  ,
        11.5130587 , 11.5755157 ],
       [26.51587619, 38.68803619, 33.52371746, ..., 47.448307  ,
        11.5521655 , 11.459198  ],
       [25.80545905, 38.42890667, 37.67869333, ..., 47.515667  ,
        11.5371056 , 11.486474  ],
       ...,
       [27.25748598, 32.62649844,  5.00346978, ..., 43.981438  ,
        11.615614  , 11.571474  ],
       [26.2822405 , 31.75218318,  5.00226231, ..., 43.87454   ,
        11.4947977 , 11.54208   ],
       [24.97500062, 31.03079439,  5.0468    , ..., 44.46934   ,
        11.459312  , 11.524663  ]])

In [255]:
# 使用XGBoost模型预测尺寸1的数据

In [256]:
model_xgb1 = xgb.XGBRegressor(max_depth=10,      #树的最大深度
                             learning_rate=0.05, #学习率
                             n_estimators=180,   #迭代次数
                             subsample=0.6,      #用于训练模型的子样本占整个样本集合的比例
                             colsample_bytree=0.8,#在建立树时对特征随机采样的比例
                             min_child_weight=3, #孩子节点中最小的样本权重和
                             reg_lambda=2,       #L2 正则的惩罚系数
                             seed=1000)          #随机数的种子

In [257]:
model_xgb1.fit(x_train, y_train[:,0])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=10,
             min_child_weight=3, missing=nan, monotone_constraints='()',
             n_estimators=180, n_jobs=0, num_parallel_tree=1, random_state=1000,
             reg_alpha=0, reg_lambda=2, scale_pos_weight=1, seed=1000,
             subsample=0.6, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [258]:
y_pred1 = model_xgb1.predict(x_test)

# 预测 尺寸3

In [259]:
# 筛选出x_train特征

In [260]:
skb=SelectKBest(f_regression, k=20)

In [261]:
x_train=skb.fit_transform(x_train,y_train[:,0])

In [262]:
# 筛选出x_test特征

In [263]:
mask=skb.get_support(indices=True)

In [264]:
x_test=x_test[:,mask]

In [265]:
# 使用XGBoost模型预测尺寸3的数据

In [266]:
model_xgb3 = xgb.XGBRegressor(max_depth=8, #树的最大深度
                             learning_rate=0.05, #学习率
                             n_estimators=180,#迭代次数
                             subsample=0.6,#用于训练模型的子样本占整个样本集合的比例
                             colsample_bytree=0.6,#在建立树时对特征随机采样的比例
                             min_child_weight=3,#孩子节点中最小的样本权重和
                             reg_lambda=2,#L2 正则的惩罚系数
                             seed=1000)#随机数的种子

In [267]:
model_xgb3.fit(x_train, y_train[:,2])
y_pred3 = model_xgb3.predict(x_test)

# 预测尺寸2

# 创建预测size2的数据集

In [268]:
#训练集数据处理
TRAIN_ZIP=zipfile.ZipFile('任务A\任务A数据集\训练集\传感器高频数据.zip')  # 读取传感器高频数据
file_list_train=TRAIN_ZIP.namelist()
df_train_spc = pd.read_csv('任务A\任务A数据集\训练集\data_spc.csv')  # 读取成型机状态数据

In [269]:
model_mean_matrix = []
for file in file_list_train:  # 计算传感器高频数据各个特征的平均值
    model_data = pd.read_csv(TRAIN_ZIP.open(file))
    model_data = np.array(model_data)
    model_data_mean = np.mean(model_data, axis=0)
    model_mean_matrix.append(model_data_mean)
model_mean_matrix = np.around(model_mean_matrix, 8)

columns_name = ['SampleTime', 'Phase', 'Sensor1', 'Sensor2', 'Sensor3', 'IJ', 'Sensor5',
       'Sensor6', 'MouldTemp1', 'MouldTemp2', 'MouldTemp3', 'MouldTemp4',
       'MouldTemp5', 'MouldTemp9', 'MouldTemp10', 'MouldTemp11', 'MouldTemp12',
       'MouldTemp13', 'MouldTemp14', 'Sensor8', 'MouldFlow1', 'MouldFlow2', 'MouldFlow3' , 'SP']
for i in range(len(columns_name)):
    columns_name[i] = columns_name[i] + '_mean'
df_train_sensor_meean = pd.DataFrame(model_mean_matrix,columns = columns_name)  # 对各个特征进行重新命名
df_train_sensor_meean.to_csv('尺寸2数据/x_train_mean.csv')
drop_means_columns = ['SampleTime_mean', 'MouldFlow1_mean', 'MouldFlow2_mean', 'MouldFlow3_mean']
for col in drop_means_columns:
    df_train_sensor_meean.drop(col, axis=1, inplace=True)  # 丢弃传感器高频数据平均值的个别特征


In [270]:
drop_spc_columns = ['EL_IV_END_TIME', 'EL_IDLE_TIME', 'EL_CORE_IN_TIME_0', 'EL_CORE_IN_TIME_1', 'EL_CORE_IN_TIME_2','EL_CORE_IN_TIME_3','EL_CORE_OUT_TIME_0',
               'EL_CORE_OUT_TIME_1', 'EL_CORE_OUT_TIME_2', 'EL_CORE_OUT_TIME_3', 'EL_SB1_TIME', 'EL_CF_TIME', 'EL_CB_TIME', 'op_mode', 'ext_mold_time_flt_0',
               'EL_VPS_TIME', 'ext_mold_time_flt_1', 'ext_mold_time_flt_2','ext_mold_time_flt_3', 'ext_mold_time_flt_4', 'EL_GET_PART_TIME', 'EL_BARREL_MEAN_5',
               'EL_BARREL_MEAN_6', 'EL_BARREL_MEAN_7', 'EL_HTRN_TI_0', 'EL_HTRN_TI_1', 'EL_HTRN_TI_2', 'EL_HTRN_TI_3', 'EL_HTRN_TI_4', 'EL_HTRN_TI_5', 'EL_HTRN_TI_6',
               'EL_HTRN_TI_7', 'ext_mold_temp_flt_4', 'ext_mold_temp_flt_5', 'ext_mold_temp_flt_6', 'ext_mold_temp_flt_7', 'spcTime', 'rob_enbl', 'remark',
               'ext_mold_flow_flt_0', 'EL_PP_STG_POS_3', 'EL_PP_STG_POS_4', 'EL_CF_END_PRESS', 'EL_PLST_BP_2', 'EL_PLST_BP_3', 'EL_PLST_BP_4']
for col in drop_spc_columns:
    df_train_spc.drop(col, axis=1, inplace=True)  # 丢弃成型机转态数据的一部分特征

In [271]:
df_x_train = pd.concat([df_train_spc, df_train_sensor_meean], axis = 1)  # 将处理完成的传感器高频数据特征与成形机状态数据进行拼接
TRAIN_ZIP.close()  # 读取传感器高频数据关闭
# df_x_train.to_csv('尺寸2数据/x_train整理.csv')
print(df_x_train)

           Id  EL_CYC_TIME  EL_PLST_TIME  EL_LAST_COOL_TIME  EL_MC_TIME  \
0       33722       37.376         6.252              5.748       3.101   
1       34803       37.362         6.762              5.238       3.093   
2       36621       37.376         6.254              5.746       3.106   
3       36622       37.372         6.605              5.395       3.098   
4       36623       37.360         6.207              5.793       3.090   
...       ...          ...           ...                ...         ...   
16595  150696       41.179         6.867              9.133       3.161   
16596  150701       41.179         7.476              8.524       3.170   
16597  150706       41.189         7.180              8.820       3.163   
16598  150711       41.199         7.531              8.469       3.160   
16599  150716       41.291         7.418              8.582       3.176   

       EL_MO_TIME  EL_EF_TIME  EL_EB_TIME  EL_IV_TIME  EL_PP_TIME  ...  \
0           1.969       1

In [272]:
#测试集数据处理
TEST_ZIP=zipfile.ZipFile('任务A\任务A数据集\测试集\传感器高频数据.zip')  # 读取传感器高频数据
file_list_test=TEST_ZIP.namelist()
df_test_spc = pd.read_csv('任务A\任务A数据集\测试集/data_spc.csv')  # 读取成型机状态数据

In [273]:
model_mean_matrix = []
for file in file_list_test:  # 计算传感器高频数据各个特征的平均值
    model_data = pd.read_csv(TEST_ZIP.open(file))
    model_data = np.array(model_data)
    model_data_mean = np.mean(model_data, axis=0)
    model_mean_matrix.append(model_data_mean)
model_mean_matrix = np.around(model_mean_matrix, 8)

columns_name = ['SampleTime', 'Phase', 'Sensor1', 'Sensor2', 'Sensor3', 'IJ', 'Sensor5',
       'Sensor6', 'MouldTemp1', 'MouldTemp2', 'MouldTemp3', 'MouldTemp4',
       'MouldTemp5', 'MouldTemp9', 'MouldTemp10', 'MouldTemp11', 'MouldTemp12',
       'MouldTemp13', 'MouldTemp14', 'Sensor8', 'MouldFlow1', 'MouldFlow2', 'MouldFlow3' , 'SP']
for i in range(len(columns_name)):
    columns_name[i] = columns_name[i] + '_mean'  
df_test_sensor_meean = pd.DataFrame(model_mean_matrix,columns = columns_name)  # 对各个特征进行重新命名

drop_means_columns = ['SampleTime_mean', 'MouldFlow1_mean', 'MouldFlow2_mean', 'MouldFlow3_mean']
for col in drop_means_columns:
    df_test_sensor_meean.drop(col, axis=1, inplace=True)  # 丢弃传感器高频数据平均值的个别特征


In [274]:
drop_spc_columns = ['EL_IV_END_TIME', 'EL_IDLE_TIME', 'EL_CORE_IN_TIME_0', 'EL_CORE_IN_TIME_1', 'EL_CORE_IN_TIME_2','EL_CORE_IN_TIME_3','EL_CORE_OUT_TIME_0',
               'EL_CORE_OUT_TIME_1', 'EL_CORE_OUT_TIME_2', 'EL_CORE_OUT_TIME_3', 'EL_SB1_TIME', 'EL_CF_TIME', 'EL_CB_TIME', 'op_mode', 'ext_mold_time_flt_0',
               'EL_VPS_TIME', 'ext_mold_time_flt_1', 'ext_mold_time_flt_2','ext_mold_time_flt_3', 'ext_mold_time_flt_4', 'EL_GET_PART_TIME', 'EL_BARREL_MEAN_5',
               'EL_BARREL_MEAN_6', 'EL_BARREL_MEAN_7', 'EL_HTRN_TI_0', 'EL_HTRN_TI_1', 'EL_HTRN_TI_2', 'EL_HTRN_TI_3', 'EL_HTRN_TI_4', 'EL_HTRN_TI_5', 'EL_HTRN_TI_6',
               'EL_HTRN_TI_7', 'ext_mold_temp_flt_4', 'ext_mold_temp_flt_5', 'ext_mold_temp_flt_6', 'ext_mold_temp_flt_7', 'spcTime', 'rob_enbl', 'remark',
               'ext_mold_flow_flt_0', 'EL_PP_STG_POS_3', 'EL_PP_STG_POS_4', 'EL_CF_END_PRESS', 'EL_PLST_BP_2', 'EL_PLST_BP_3', 'EL_PLST_BP_4']
for col in drop_spc_columns:
    df_test_spc.drop(col, axis=1, inplace=True)  # 丢弃成型机转态数据的一部分特征

In [275]:
df_x_test = pd.concat([df_test_spc, df_test_sensor_meean], axis = 1)  # 将处理完成的传感器高频数据特征与成形机状态数据进行拼接
TEST_ZIP.close()  # 读取传感器高频数据关闭
# df_x_test.to_csv('尺寸2数据/x_test整理.csv')
print(df_x_test)

          Id  EL_CYC_TIME  EL_PLST_TIME  EL_LAST_COOL_TIME  EL_MC_TIME  \
0      56694       37.609         7.305              4.695       3.090   
1      56699       37.620         7.085              4.915       3.094   
2      56705       37.615         7.067              4.933       3.098   
3      56708       37.622         7.254              4.746       3.099   
4      56712       37.621         7.109              4.891       3.100   
...      ...          ...           ...                ...         ...   
3966  175941       39.274         7.932              7.068       2.972   
3967  175956       39.214         7.912              7.088       2.958   
3968  175971       39.223         7.903              7.097       2.967   
3969  176121       39.203         6.765              8.235       2.957   
3970  176136       39.229         7.764              7.236       2.960   

      EL_MO_TIME  EL_EF_TIME  EL_EB_TIME  EL_IV_TIME  EL_PP_TIME  ...  \
0          1.963       1.051       1.9

In [276]:
df_y_train = pd.read_csv('任务A\任务A数据集\训练集\Size.csv')
df_x_train.drop('Id', axis=1, inplace=True)
df_x_test.drop('Id', axis=1, inplace=True)
df_y_train.drop('Id', axis=1, inplace=True)

    
x_train = np.array(df_x_train)
y_train = np.array(df_y_train)
x_test = np.array(df_x_test)

In [277]:
# 使用lightGBM模型预测尺寸2的数据

In [278]:
model_lgb = lgb.LGBMRegressor(boosting_type='gbdt',  # 设置提升类型
                              objective='regression',  # 目标函数
                              #     metric= 'l2',  # 评估函数
                              num_leaves=25,  # 叶子节点数
                              learning_rate=0.24,  # 学习速率
                              feature_fraction=0.9,  # 建树的特征选择比例
                              bagging_fraction=0.8,  # 建树的样本采样比例
                              bagging_freq=5,  # k 意味着每 k 次迭代执行bagging
                              verbose=1)  # <0 显示致命的, =0 显示错误 (警告), >0 显示信息)

In [279]:
model_lgb .fit(x_train, y_train[:,1])
y_pred2 = model_lgb .predict(x_test)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007298 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12418
[LightGBM] [Info] Number of data points in the train set: 16600, number of used features: 59
[LightGBM

In [280]:
 x_train.shape

(16600, 59)

# 结合

In [281]:
y_pred=np.zeros((3971,3))
y_pred[:,0]=y_pred1
y_pred[:,1]=y_pred2
y_pred[:,2]=y_pred3

In [282]:
df = pd.DataFrame(y_pred)
df.to_csv('subfile000.csv')